# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.risk.linalg import pca
from cvx.risk.factor import FactorModel

## Load prices and compute returns

In [2]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

## Compute principal components

In [3]:
factors = pca(returns=returns, n_components=10)

## Create the risk model, here a FactorModel

In [5]:
model = FactorModel(assets=len(returns.columns), k=10)

# update the model parameters
model.update(cov=factors.cov, exposure=factors.exposure.values, idiosyncratic_risk=factors.idiosyncratic.std().values)

# test the risk model with uniform weights
weights = 0.05 * np.ones(20)
model.estimate(weights).value

0.00923407730537884

## RiskModel is injected into optimizer

In [8]:
w = cvx.Variable(20)
y = cvx.Variable(10)

objective = cvx.Minimize(1e3*model.estimate(w, y=y))
constraints = [w >= 0, cvx.sum(w) == 1] + model.bounds.constraints(w)

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate(w, y=y).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

GOOG    0.000841
AAPL    0.000463
FB      0.000462
BABA    0.003246
AMZN    0.000594
GE      0.003120
AMD     0.143065
WMT     0.001252
BAC     0.001290
GM      0.000356
T       0.000359
UAA     0.186590
SHLD    0.534772
XOM     0.000519
RRC     0.036929
BBY     0.082946
MA      0.000675
PFE     0.000576
JPM     0.001600
SBUX    0.000344
dtype: float64
0.000170523449914986
